In [1]:
import numpy as np
import cv2 as cv


# read names of classes
with open('synset_words.txt') as f:
    classes = [x[x.find(' ') + 1:] for x in f]

image = cv.imread('duck07.png')
# create tensor with 224x224 spatial size and subtract mean values (104, 117, 123) 
# from corresponding channels (R, G, B)
input = cv.dnn.blobFromImage(image, 1, (224, 224), (104, 117, 123))

# load model from caffe
net = cv.dnn.readNetFromCaffe(
    'deploy.prototxt',
    'bvlc_googlenet.caffemodel'
)
# feed input tensor to the model
net.setInput(input)
# perform inference and get output
out = net.forward()
# get indices with the highest probability
indexes = np.argsort(out[0])[-5:]
for i in reversed(indexes):
    print('class:', classes[i], ' probability:', out[0][i])

class: drake
  probability: 0.9999695
class: red-breasted merganser, Mergus serrator
  probability: 2.9070698e-05
class: goose
  probability: 7.0552625e-07
class: American coot, marsh hen, mud hen, water hen, Fulica americana
  probability: 6.5966043e-07
class: albatross, mollymawk
  probability: 4.7360803e-08


In [11]:
import numpy as np

# Укажите пути к файлам вручную
image_path = "duck07.png"  # Путь к изображению
confidence_threshold = 0.2  # Минимальная уверенность для фильтрации слабых детекций

# Классы, которые может распознавать MobileNet SSD
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# Генерация случайных цветов для рамок
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# Загрузка модели
print("[INFO] Загружается модель...")
new_net = cv.dnn.readNetFromCaffe(
    'mobilenet_deploy.prototxt',
    'mobilenet_iter_73000.caffemodel'
)

# Загрузка изображения
image = cv.imread(image_path)
(h, w) = image.shape[:2]

# Преобразование изображения в формат, подходящий для нейросети
blob = cv.dnn.blobFromImage(
    cv.resize(image, (300, 300)),
    0.007843,
    (300, 300),
    127.5
)

# Обнаружение объектов
print("[INFO] Обнаружение объектов...")
new_net.setInput(blob)
detections = new_net.forward()

# Обработка детекций
for i in np.arange(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > confidence_threshold:
        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
        print("[INFO]", label)
        
        cv.rectangle(
            image,
            (startX, startY),
            (endX, endY),
            COLORS[idx],
            2
        )
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv.putText(
            image,
            label,
            (startX, y),
            cv.FONT_HERSHEY_SIMPLEX,
            0.5,
            COLORS[idx],
            2
        )

# Отображение результата
cv.imshow("Output", image)
cv.waitKey(0)
cv.destroyAllWindows()

[INFO] Загружается модель...
[INFO] Обнаружение объектов...
[INFO] bird: 100.00%
